# 1- Packages

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from scipy import stats
from textwrap import wrap
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


%matplotlib inline

# 2 - Data acquisition

In [680]:
universe=pd.read_excel(r'C:\Users\Eric.Li\Documents\Post result data\China price 2007-2017.xlsx',sheet_name="By year")

XLRDError: No sheet named <'Universe'>

In [615]:
Europe=pd.read_excel(r'C:\Users\Eric.Li\Documents\IPO data\IPO.xlsx',sheet_name="Europe").set_index("Ticker")
price_Europe=pd.read_excel(r'C:\Users\Eric.Li\Documents\IPO data\IPO.xlsx',sheet_name="Europe price").set_index("Ticker")
index_Europe=pd.read_excel(r'C:\Users\Eric.Li\Documents\IPO data\IPO.xlsx',sheet_name="Europe index").set_index("Ticker")

In [616]:
def get_return(price_df,index_df):
    abs_return=price_df.diff(1,axis=1)/price_df.shift(1,axis=1)
    index_return=index_df.diff(1,axis=1)/index_df.shift(1,axis=1)
    rel_return=abs_return-index_return
    return abs_return,rel_return

In [617]:
abs_return_Europe,rel_return_Europe=get_return(price_Europe,index_Europe)

# 3 - Strategy

In [628]:
def IPO_long(data_df,price_df,return_df,Size,Open,Close,Volume,liquidity_constraint,start_day,end_day,holding):
    target_data=data_df[(data_df["Market_cap"]>=Size)&(data_df["Offer_Day1_Open"]>Open)&(data_df["Offer_Day1_Close"]>Close)&(data_df["Volume_shares"]>Volume)&(data_df["5d_liquidity"]>liquidity_constraint)]
    target_price=price_df.loc[target_data.index.tolist()]
    high_evaluate=(target_price.expanding(min_periods=start_day,axis=1).max()==target_price).iloc[:,start_day-1:end_day]
    high_evaluate=high_evaluate[high_evaluate==True].dropna(how='all',axis=0)
    trade_day=high_evaluate.apply(lambda x:x.tolist().index(1.0),axis=1)    
    target_return=return_df.loc[trade_day.index]
    signal_df=target_return.apply(lambda x:x[5-1+trade_day[x.name]:5-1+trade_day[x.name]+holding],axis=1).shift(1,axis=1)
    signal_df["Index"]=signal_df.apply(lambda x:(x.name,data_df["IPO_date"].loc[x.name].date(),trade_day[x.name]+6),axis=1)
    signal_df=signal_df.set_index("Index")
    return signal_df

In [676]:
signal_Europe=IPO_long(Europe,price_Europe,rel_return_Europe,300,0,0,0,3,5,80,40)

In [677]:
signal_Europe.sum(axis=1).mean()

0.0702851526377321

In [678]:
(signal_Europe.sum(axis=1)>0).sum()

58

In [679]:
signal_Europe.shape

(80, 110)

In [627]:
signal_Europe

,5,6,7,8,9,10,11,12,13,14,...,67,68,69,70,71,72,73,74,75,76
Index,,,,,,,,,,,,,,,,,,,,,
"(AENA SM Equity, 2015-02-11, 6)",NaN,0.031606,0.018345,-0.002787,0.010316,-0.011272,-0.011290,0.009629,0.022763,0.023280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(CLNX SM Equity, 2015-05-07, 6)",NaN,0.032011,0.005598,-0.017237,-0.012303,0.004870,-0.000361,-0.006517,-0.022294,0.001198,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(RMG LN Equity, 2013-10-11, 7)",NaN,NaN,0.039260,-0.007418,-0.003646,0.085546,-0.022123,0.050617,-0.011209,-0.031422,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(SLR SM Equity, 2007-06-19, 26)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(MONC IM Equity, 2013-12-16, 12)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.074542,-0.002602,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(MAIL LI Equity, 2010-11-05, 11)",NaN,NaN,NaN,NaN,NaN,NaN,0.035346,0.002009,0.012060,0.040379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(GALE SW Equity, 2017-04-07, 24)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(AUTO LN Equity, 2015-03-19, 15)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(EKT SM Equity, 2015-07-01, 18)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
